텍스트 RAG 시스템 구축 코드 분석


In [ ]:
1. 필수 라이브러리 설치 (코랩/Jupyter 환경)
!pip install koreanize_matplotlib # Matplotlib 한글 지원
!pip install sentence-transformers # 텍스트를 벡터로 변환하는 SOTA 모델 제공 (임베딩)
!pip install chromadb # 벡터를 저장하고 검색하는 데이터베이스

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 23.5 MB/s et

In [ ]:
import os, re, uuid   # uuid : 고유 식별 id 생성용 / 저장할 텍스트 덩어리(chunk)마다 고유 ID를 부여하는 데 사용
from typing import List # type hint 기능 제공(가독성)
from sentence_transformers import SentenceTransformer # 임베딩 모델
from sklearn.metrics.pairwise import cosine_similarity  # 두 벡터의 유사도를 측정 (직접 사용되진 않지만 원리상 핵심)
from chromadb import PersistentClient # ChromaDB 클라이언트 (데이터를 디스크에 영구 저장)
import koreanize_matplotlib

TXT_PATH = "sample.txt"
CHROMA_DIR = ".chroma_txt_demo"
COLLECTION = "docs"
MODEL_NAME = "all-MiniLM-L6-v2" # SentenceTransformer에서 제공하는 경량화 임베딩 모델

model = SentenceTransformer(MODEL_NAME) # 임베딩 모델 로드 (트랜스포머 기반)
client = PersistentClient(path = CHROMA_DIR)  # ChromaDB 클라이언트 생성 (데이터 영구 저장)
collection = client.get_or_create_collection(COLLECTION)  # 컬렉션 생성/가져오기

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 기능에 따른 함수 작성

# 텍스트 파일 읽기 함수
def read_textFunc(path:str) -> str:
  if not os.path.exists(path):
    raise FileNotFoundError(f"파일이 없어요. {path}")
  with open(path, 'r', encoding='utf-8', errors='ignore') as f: # ... 파일 존재 여부 확인 및 utf-8로 내용 전체 읽기
    return f.read()

# 문단 분리 함수 (Chunking, RAG의 첫 단계)
def split_paragraphFunc(text:str, min_len:int=20) -> List[str]:
  paras = re.split(r'\n\s*\n+', text)   # # 1. 정규 표현식으로 '빈 줄'을 기준으로 문단을 분리. (Chunking)
  paras = [re.sub(r"\s+", "", p).strip() for p in paras]   # 공백 제거
  paras = [p for p in paras if len(p) >= min_len]  # 2. 각 문단의 공백을 제거 후 최소 길이(20자) 미만인 문단은 버림.
  return paras

# 텍스트를 벡터로 변환 (Embedding)
def embedFunc(texts:List[str]) -> List[List[float]]:  # SentenceTransformer 모델을 사용해 텍스트를 벡터로 변환.
  return model.encode(texts, normalize_embeddings=True).tolist()  # normalize_embeddings=True: 벡터 크기(L2 Norm)를 1로 정규화. (⭐매우 중요!)

# text 파일을 읽어 vectorDB에 저장 (Upsert)
def upsert_paragraphFunc(source_path:str):
  # 1. 전처리 및 벡터화
  text = read_textFunc(source_path) # 1. 텍스트 읽고 문단 분리 (Chunking)
  print(text)
  chunks = split_paragraphFunc(text)
  print(chunks)
  if not chunks:
    print('저장할 문단이 없어요')
    return
  # 2. 고유 ID 생성 (UUID)
  ids = [str(uuid.uuid4()) for _ in chunks] # ChromaDB의 Primary Key 역할
  print('ids :' , ids)
  # 3. 임베딩 벡터 생성
  embs = embedFunc(chunks)
  print('embs :' , embs)
  # 4. 메타데이터 생성
  metas = [{"source":os.path.basename(source_path), "len":len(c)}for c in chunks] # 파일명, 길이 등의 정보 추가
  # 5. ChromaDB에 저장 (add)
  collection.add(ids=ids, documents=chunks, embeddings=embs, metadatas=metas)
  print(f'저장 완료 : {len(chunks)}개 문단')

def searchFunc(query:str, k:int):
  # 1. 쿼리(질문) 임베딩: 질문도 DB의 텍스트와 같은 방식으로 벡터화
  q_emb = embedFunc([query])
  # 2. ChromaDB 쿼리: 가장 유사한 k개 결과를 검색
  res = collection.query(query_embeddings=q_emb, n_results=k)
  # 3. 결과 추출 및 출력: 문서, 메타데이터, 거리(Distance) 정보를 확인
  docs = res.get('documents', [[]])[0]    # 실제 텍스트 내용
  metas = res.get('metadatas', [[]])[0]   # 메타데이터 (파일명, 길이)
  ids = res.get('ids', [[]])[0]
  dists = res.get('distances', [[]])[0]   # 쿼리 벡터와의 거리 (작을수록 유사)

  for i, (doc, meta, _id, dist) in enumerate(zip(docs, metas, ids, dists)):
    print(f'\n[{i}] id={_id}')
    print(f'source={meta.get('source')}, len={meta.get("len")}')
    print(f'dist={dist:.4f}')
    print(doc[:300] + ("..." if len(doc) > 300 else ""))

핵심내용

Chunking (문단 분리) : RAG 시스템의 가장 중요한 전처리 단계. 문서를 작은 덩어리(청크)로 나누어야 LLM의 Context Window에 맞출 수 있고, 검색 정확도가 높아짐. 여기서는 빈 줄을 기준

normalize_embeddings=True  :  벡터 정규화. 코사인 유사도 계산이 내적으로 단순화되어 검색 속도와 안정성이 향상.

uuid.uuid4(): 텍스트 청크마다 고유한 식별자를 부여하여 DB 관리를 용이

쿼리 임베딩 : 질문(query) 자체를 임베딩하여 DB에 저장된 텍스트 벡터들과 비교합니다. 이는 의미적 유사성을 측정하는 핵심

유사도 검색 : query 함수가 내부적으로 코사인 유사도를 계산하여 쿼리 벡터와 가장 가까운 상위 K개 청크를 찾아냄. (이 K개 청크가 LLM에게 제공될 문맥 정보임.)

⭐꼭 알아두기!⭐

Sentence-Transformers = 텍스트 임베딩: SentenceTransformer는 텍스트 RAG의 핵심. 텍스트를 의미 있는 벡터로 변환하는 트랜스포머 기반의 도구.

RAG의 3요소: Chunking (분리) → Embedding (벡터화) → Retrieval (검색) 순서가 텍스트 RAG 파이프라인의 기본.

벡터 정규화: normalize_embeddings=True는 필수. 정확하고 빠른 유사도 검색(내적)을 위한 기반.

ChromaDB의 역할: 문단(Documents), 벡터(Embeddings), 고유 ID(IDs), 부가 정보(Metadatas) 네 가지를 묶어서 관리하는 저장소.


In [ ]:
if __name__ =="__main__":
  upsert_paragraphFunc(TXT_PATH)
  print("\n검색 예 :")
  searchFunc("셧다운", k=3)



이데일리 김윤지 기자] 미국 하원이 12일(현지시간) 상원이 통과시킨 셧다운(일시적 업무 중지) 종료를 위한 합의안을 최종 가결했다. 임시예산안이 하원을 최종 통과하면서 도널드 트럼프 대통령의 서명만 남았다.

미 하원은 이날 오후 전체회의에서 단기 지출법안(CR·임시예산안)에 대한 표결을 실시해 이를 통과시켰다.
백악관은 앞서 트럼프 대통령이 해당 임시예산안을 지지한다면서 이날 오후 9시 45분(한국시간 13일 오전 11시 45분) 서명식을 진행한다고 밝혔다.

표결에 앞서 민주당 하원 원내대표 하킴 제프리스(뉴욕)는 “이 싸움이 끝나는 방식은 단 두 가지뿐이다. 올해 공화당이 마침내 ‘오바마케어‘ 세액공제를 연장하기로 결정하든지, 아니면 미국 국민이 내년에 공화당을 그들의 자리에서 몰아내고 도널드 트럼프의 대통령직을 영원히 끝내든지다”라고 말했다.

이어 마이크 존슨(공화·루이지애나) 하원의장은 “이 모든 과정은 완전히 무의미했다. 잘못된 것이었고, 잔혹했다“며 ”우리는 더 이상 미국인들을 위해 시간을 낭비할 수 없다“고 말했다.
['이데일리김윤지기자]미국하원이12일(현지시간)상원이통과시킨셧다운(일시적업무중지)종료를위한합의안을최종가결했다.임시예산안이하원을최종통과하면서도널드트럼프대통령의서명만남았다.', '미하원은이날오후전체회의에서단기지출법안(CR·임시예산안)에대한표결을실시해이를통과시켰다.백악관은앞서트럼프대통령이해당임시예산안을지지한다면서이날오후9시45분(한국시간13일오전11시45분)서명식을진행한다고밝혔다.', '표결에앞서민주당하원원내대표하킴제프리스(뉴욕)는“이싸움이끝나는방식은단두가지뿐이다.올해공화당이마침내‘오바마케어‘세액공제를연장하기로결정하든지,아니면미국국민이내년에공화당을그들의자리에서몰아내고도널드트럼프의대통령직을영원히끝내든지다”라고말했다.', '이어마이크존슨(공화·루이지애나)하원의장은“이모든과정은완전히무의미했다.잘못된것이었고,잔혹했다“며”우리는더이상미국인들을위해시간을낭비할수없다“고말했다.']
ids : ['bc2f483b-38a5-495d-86b8-24f82a